In [1]:
# calcul du plus long chemin entre deux sommmets d'un graphe
# Input: Graph G (adjacency matrix) 
#        Source vertex s
function BellmanFord2(G, s)
    nbv = size(G)[1]
    # dans chaque ligne i  
    # dis[i, 1] correspond au cout min entre s et le noeud i 
    # dis[i, 2] correspond au predecesseur par lequel on trouve le cout min
    dis = zeros(nbv, 2) 
    for i in 1:nbv
        dis[i, 1] = -Inf
        dis[i, 2] = -1
    end
    dis[s, 1] = 0
    for k in 1:(nbv - 1)
        # edge (i, j)
        for i in 1:nbv
            for j in 1:nbv
                if (G[i, j] != -1) && (dis[j, 1] < (dis[i, 1] + G[i, j]))
                    dis[j, 1] = dis[i, 1] + G[i,j]
                    dis[j, 2] = i
                end
            end
        end
    end
    return dis
end

BellmanFord2 (generic function with 1 method)

In [2]:
function affres2(dis, s)
   println("Result :")
   println("node     | biggest cost from $s      | previous node")
   println("")
   for i in 1:size(dis)[1]
        println("$i           $(dis[i, 1])                          $(dis[i, 2])")
    end
end

affres2 (generic function with 1 method)

In [3]:
# 1.2 Modélisation classique par graphe potentiel-tache
# Graphe :
G = [ -1  2  2 -1 -1 -1;
      -1 -1 -1  3 -1 -1;
      -1 -1 -1  1  1 -1;
      -1 -1 -1 -1 -1  4;
      -1 -1 -1 -1 -1  1;
      -1 -1 -1 -1 -1 -1;
];
dis = BellmanFord2(G, 1);
affres2(dis, 1)

Result :
node     | biggest cost from 1      | previous node

1           0.0                          -1.0
2           2.0                          1.0
3           2.0                          1.0
4           5.0                          2.0
5           3.0                          3.0
6           9.0                          4.0


In [4]:
# 2.1 Relaxation des contraintes de ressources
D = transpose([6 7 0; 3 5 1])


3×2 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 6  3
 7  5
 0  1

In [5]:
# solution lineaire 
using JuMP
using Clp
model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:3, j in 1:2] >= 0)
@variable(model, tfin1)
@variable(model, tfin2)

# define objective function
@objective(model, Min, (tfin1+tfin2))

# define constraints
@constraint(model, t[2,1] - t[1,1] >= D[1,1] )
@constraint(model, t[3,1] - t[2,1] >= D[2,1] )
@constraint(model, t[2,2] - t[1,2] >= D[1,2] )
@constraint(model, t[3,2] - t[2,2] >= D[2,2] )

@constraint(model, tfin1 - t[3,1] >= D[3,1] )
@constraint(model, tfin2 - t[3,2] >= D[3,2] )

println(model)

print("start solve \n")
optimize!(model)
print("end solve\n")

println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", max(value(tfin2), value(tfin1)))

Min tfin1 + tfin2
Subject to
 -t[1,1] + t[2,1] ≥ 6.0
 -t[2,1] + t[3,1] ≥ 7.0
 -t[1,2] + t[2,2] ≥ 3.0
 -t[2,2] + t[3,2] ≥ 5.0
 -t[3,1] + tfin1 ≥ 0.0
 -t[3,2] + tfin2 ≥ 1.0
 t[1,1] ≥ 0.0
 t[2,1] ≥ 0.0
 t[3,1] ≥ 0.0
 t[1,2] ≥ 0.0
 t[2,2] ≥ 0.0
 t[3,2] ≥ 0.0

start solve 
end solve


Solution PL:
 	 t=[0.0 0.0; 6.0 3.0; 13.0 8.0]	 tfin=13.0


In [11]:
# Graphe potentiel-tâche

# Explication des indices de G:
# 1 :: Etat initial
# 2 3 4 :: t[1,1] t[2,1] t[3,1]
# 5 6 7 :: t[1,2] t[2,2] t[3,3]
# 8 :: Etat final

G2 = [ -1 0 -1 -1  0 -1 -1 -1;
      -1 -1  6 -1 -1 -1 -1 -1;
      -1 -1 -1  7 -1 -1 -1 -1;
      -1 -1 -1 -1 -1 -1 -1  0;
      -1 -1 -1 -1 -1  3 -1 -1;
      -1 -1 -1 -1 -1 -1  5 -1;
      -1 -1 -1 -1 -1 -1 -1  1;
      -1 -1 -1 -1 -1 -1 -1 -1;
];
dis = BellmanFord2(G2, 1);
affres2(dis, 1)

println("\n\nSolution Graphe potentiel-tâche:\t   $(dis[8, 1])")

Result :
node     | biggest cost from 1      | previous node

1           0.0                          -1.0
2           0.0                          1.0
3           6.0                          2.0
4           13.0                          3.0
5           0.0                          1.0
6           3.0                          5.0
7           8.0                          6.0
8           13.0                          4.0


Solution Graphe potentiel-tâche:	   13.0
